In [31]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [32]:
def _parse_line(line):

    rx_dict = {
    'token': re.compile(r'var token = "(?P<token>.*)"\n'),
    'url': re.compile(r'var url = "(?P<url>.*)"\n'),
    'org': re.compile(r'var org = "(?P<org>.*)"\n'),
    'bucket': re.compile(r'var bucket = "(?P<bucket>.*)"\n'),
    }   

    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

filepath = '/root/flexi-pipe/config.go'
# open the file and read through it line by line
with open(filepath, 'r') as file_object:
    line = file_object.readline()
    while line:
        # at each line check for a match with a regex
        key, match = _parse_line(line)

        if key == 'token':
            token = match.group('token')
        elif key == 'url':
            url = match.group('url')
        elif key == 'org':
            org = match.group('org')
        elif key == 'bucket':
            bucket = match.group('bucket')
        
        line = file_object.readline()
# url="http://192.168.20.58:8086"
url = "http://localhost:8086"

In [33]:
start = 1696343921
end = 1696710919

In [34]:
def experiment(start_time, end_time, filepath):
    # Retrieve experiments data from csv
    data = pd.read_csv(filepath, header=None)
    df = pd.DataFrame(data)

    #Rename columns
    experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

    #Correct timestamp
    experiments["start"] = experiments["start"].str.slice(0, 27)
    experiments["end"] = experiments["end"].str.slice(0, 27)

    #String to timestamp
    # experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    # experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
    experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


    experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
    experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

    #Drop fields we don't mneed for the moment
    exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

    #Get times for different intervals
    # intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
    # intervals.head(10)

    expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
    # expTime['experiment'] = expTime.index
    expTime = expTime.reset_index().rename({'index':'experiment'}, axis = 'columns')

    return expTime

experiments = experiment(start, end, '../experiments.csv')
experiments.head(5)

/tmp/ipykernel_4421/1055150111.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
/tmp/ipykernel_4421/1055150111.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
0,383,2023-10-03 14:38:41.3311106,2023-10-03 15:08:41.3340502,unl,reference,8,6,12,4,8,2,0.25,1.0,1696343921,1696345721
1,384,2023-10-04 12:58:35.4458617,2023-10-04 13:13:35.4494771,unl,reference,8,6,12,4,8,2,0.25,1.0,1696424315,1696425215
2,385,2023-10-04 13:25:01.0752251,2023-10-04 13:40:01.0774462,unl,reference,8,6,12,4,8,2,0.25,1.0,1696425901,1696426801
3,386,2023-10-04 14:18:48.4409855,2023-10-04 14:33:48.4430498,unl,reference,8,6,12,4,8,2,0.25,1.0,1696429128,1696430028
4,387,2023-10-04 15:38:33.0103914,2023-10-04 15:53:33.0161317,unl,reference,8,6,12,4,8,2,0.25,1.0,1696433913,1696434813


In [35]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [36]:
data = pd.read_csv('./experiments_filtered.csv', header=0,  index_col=0)
exps = pd.DataFrame(data)

exps.head(10)

,experiment,originalStart,originalEnd,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,start,end,startUnix,endUnix
3,390,2023-10-04 18:52:03.080773000,2023-10-04 19:22:03.086651800,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-10-04 18:59:33.080773000,2023-10-04 19:14:33.086651800,1696445973,1696446873
4,391,2023-10-04 19:25:37.153339900,2023-10-04 19:55:37.155585600,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-10-04 19:33:07.153339900,2023-10-04 19:48:07.155585600,1696447987,1696448887
6,393,2023-10-04 20:32:47.309861600,2023-10-04 21:02:47.314917800,unl,informed,8,6,12,4,8,2,0.25,3.0,2023-10-04 20:40:17.309861600,2023-10-04 20:55:17.314917800,1696452017,1696452917
7,394,2023-10-04 21:06:22.775563400,2023-10-04 21:36:22.778880000,unl,informed,8,6,12,4,8,2,0.25,3.0,2023-10-04 21:13:52.775563400,2023-10-04 21:28:52.778880000,1696454032,1696454932
9,396,2023-10-04 22:13:33.814670400,2023-10-04 22:43:33.817409400,unl,informed0,16,8,20,16,8,4,0.50,1.0,2023-10-04 22:21:03.814670400,2023-10-04 22:36:03.817409400,1696458063,1696458963
10,397,2023-10-04 22:47:09.383711000,2023-10-04 23:17:09.386844200,unl,informed0,16,8,20,16,8,4,0.50,1.0,2023-10-04 22:54:39.383711000,2023-10-04 23:09:39.386844200,1696460079,1696460979
15,404,2023-10-05 02:42:12.104487300,2023-10-05 03:12:12.156487200,unl,informed2,16,8,20,16,8,4,0.50,0.5,2023-10-05 02:49:42.104487300,2023-10-05 03:04:42.156487200,1696474182,1696475082
16,405,2023-10-05 03:15:44.641811200,2023-10-05 03:45:44.644068700,unl,informed3,6,4,8,2,2,2,0.25,1.0,2023-10-05 03:23:14.641811200,2023-10-05 03:38:14.644068700,1696476194,1696477094
17,406,2023-10-05 03:49:18.120429500,2023-10-05 04:19:18.122983600,unl,informed3,6,4,8,2,2,2,0.25,1.0,2023-10-05 03:56:48.120429500,2023-10-05 04:11:48.122983600,1696478208,1696479108
21,412,2023-10-05 07:10:41.394283100,2023-10-05 07:40:41.433055700,unl,informed5,6,4,8,2,2,4,0.25,0.5,2023-10-05 07:18:11.394283100,2023-10-05 07:33:11.433055700,1696490291,1696491191


In [37]:
data = pd.read_csv('./propagation.csv', header=0,  index_col=0)
final = pd.DataFrame(data)

final.head(10)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes,consensus,propagationTime,stddev_propagationTime
0,0,unl,8,6,12,4,8,2,0.25,1.0,24,3.022872,2149.216025,257.421035
1,3,unl,8,6,12,4,8,2,0.25,3.0,24,3.026471,2160.133107,126.586885
2,6,unl,16,8,20,16,8,4,0.50,1.0,24,3.027537,2008.636699,159.608146
3,12,unl,16,8,20,16,8,4,0.50,0.5,24,3.073917,6674.953667,0.000000
4,13,unl,6,4,8,2,2,2,0.25,1.0,24,3.038808,3089.497825,259.143435
5,18,unl,6,4,8,2,2,4,0.25,0.5,24,3.014395,2097.297225,201.000893
6,21,unl,20,16,24,8,16,8,0.50,1.0,24,3.029904,2013.597379,287.651372
7,23,unl,16,8,20,4,8,2,0.50,1.0,24,3.021019,2353.420496,316.162061
8,27,unl,16,8,20,8,8,4,0.25,1.0,24,3.025704,1903.031597,0.000000
9,29,unl,16,8,20,4,8,2,0.25,1.0,24,3.044116,1963.333217,48.432417


In [38]:
duplicateMessage = pd.DataFrame()

for index, row in final.iterrows():
    # print(row["start"], row["end"], row["experiment"])
    execs = exps.loc[(exps["topology"] == row["topology"]) & (exps["d"] == row["d"]) &
                            (exps["dlo"] == row["dlo"]) & (exps["dhi"] == row["dhi"]) &
                            (exps["dscore"] == row["dscore"]) & (exps["dlazy"] == row["dlazy"]) &
                            (exps["dout"] == row["dout"]) & (exps["gossipFactor"] == row["gossipFactor"]) &                    
                            (exps["interval"] == row["interval"])]

    for idx, exec in execs.iterrows():
        # print(exec["startUnix"], exec["endUnix"])
        query_mess = from_influx(url, token, org, "duplicateMessage", exec["startUnix"], exec["endUnix"],"_measurement")
    
        query_mess["identifier"] = row["identifier"]

        query_mess["experiment"] = exec["experiment"]
 
        query_mess["_time"] = pd.to_datetime(query_mess["_time"])
    
        duplicateMessage = pd.concat([query_mess, duplicateMessage])

        del query_mess

        gc.collect()

duplicateMessage.head(10)


,_time,_measurement,identifier,experiment
0,2023-10-07 17:58:32.814736+00:00,duplicateMessage,85,499
1,2023-10-07 17:58:32.814755+00:00,duplicateMessage,85,499
2,2023-10-07 17:58:32.814763+00:00,duplicateMessage,85,499
3,2023-10-07 17:58:32.814774+00:00,duplicateMessage,85,499
4,2023-10-07 17:58:32.814804+00:00,duplicateMessage,85,499
5,2023-10-07 17:58:32.814805+00:00,duplicateMessage,85,499
6,2023-10-07 17:58:32.814813+00:00,duplicateMessage,85,499
7,2023-10-07 17:58:32.814815+00:00,duplicateMessage,85,499
8,2023-10-07 17:58:32.814844+00:00,duplicateMessage,85,499
9,2023-10-07 17:58:32.814886+00:00,duplicateMessage,85,499


In [41]:
df = duplicateMessage.drop(columns=['_measurement'])

avgPropExp = df.groupby(['experiment', 'identifier']).agg({'_time':['count']})
avgPropExp.columns = avgPropExp.columns.droplevel(0)
avgPropExp.reset_index(inplace=True)
avgPropExp = avgPropExp.drop(columns=['experiment'])
avgPropExp.head(10)

avgOve = avgPropExp.groupby(['identifier']).agg({'count':['mean','std']}).fillna(0)
avgOve.columns = avgOve.columns.droplevel(0)
avgOve.reset_index(inplace=True)

avgOve.head(10)


,identifier,mean,std
0,0,1262081.5,255305.853095
1,3,1220458.0,107468.917032
2,6,991947.5,137230.334345
3,12,388920.0,0.000000
4,13,859426.5,79732.653540
5,18,391164.5,201636.448403
6,21,1063446.0,33021.886681
7,23,1347907.5,95037.272712
8,27,952511.0,0.000000
9,29,809796.5,45515.756398


In [42]:
finalOve = final.merge(avgOve, on='identifier')
finalOve = finalOve.rename(columns={'mean' : 'messageOverhead', 'std' : 'stddev_messageOverhead'})#.drop(columns=['std'])

finalOve.to_csv('overhead.csv')
finalOve.head(10)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes,consensus,propagationTime,stddev_propagationTime,messageOverhead,stddev_messageOverhead
0,0,unl,8,6,12,4,8,2,0.25,1.0,24,3.022872,2149.216025,257.421035,1262081.5,255305.853095
1,3,unl,8,6,12,4,8,2,0.25,3.0,24,3.026471,2160.133107,126.586885,1220458.0,107468.917032
2,6,unl,16,8,20,16,8,4,0.50,1.0,24,3.027537,2008.636699,159.608146,991947.5,137230.334345
3,12,unl,16,8,20,16,8,4,0.50,0.5,24,3.073917,6674.953667,0.000000,388920.0,0.000000
4,13,unl,6,4,8,2,2,2,0.25,1.0,24,3.038808,3089.497825,259.143435,859426.5,79732.653540
5,18,unl,6,4,8,2,2,4,0.25,0.5,24,3.014395,2097.297225,201.000893,391164.5,201636.448403
6,21,unl,20,16,24,8,16,8,0.50,1.0,24,3.029904,2013.597379,287.651372,1063446.0,33021.886681
7,23,unl,16,8,20,4,8,2,0.50,1.0,24,3.021019,2353.420496,316.162061,1347907.5,95037.272712
8,27,unl,16,8,20,8,8,4,0.25,1.0,24,3.025704,1903.031597,0.000000,952511.0,0.000000
9,29,unl,16,8,20,4,8,2,0.25,1.0,24,3.044116,1963.333217,48.432417,809796.5,45515.756398
